In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.linalg import inv
import folium
from folium import plugins
%matplotlib inline

In [31]:
water_tank = pd.read_csv('data/Self-Reported_Drinking_Water_Tank_Inspection_Results.csv')
fea = ['BIN','BOROUGH','ZIP','STREET_NAME','BLOCK', 'LOT','CONFIRMATION_NUM','REPORTING_YEAR',\
                              'INSPECTION_DATE','MEET_STANDARDS','LATITUDE','LONGITUDE','BBL']
water_tank_clean = water_tank[fea].dropna()
df_2020_c = water_tank_clean[water_tank_clean.REPORTING_YEAR >= 2020]
df_2020_c = df_2020_c[df_2020_c.MEET_STANDARDS == 'Y']
df_2020_c.index = np.arange(0,len(df_2020_c.index))

In [32]:
restaurant_csv_path = "data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv"
df_insp = pd.read_csv(restaurant_csv_path).copy()
df_insp = df_insp.dropna()
df_insp.index = np.arange(0,len(df_insp.index))

In [33]:
df_2020_c_sorted = df_2020_c.sort_values(by = ['INSPECTION_DATE'])
dup = df_2020_c_sorted.duplicated(subset=['LATITUDE'], keep='first')
ind = [not i for i in dup]
df_2020_c_uni = df_2020_c_sorted[ind].sort_values(by = ['INSPECTION_DATE'])
df_2020_c_uni.index = np.arange(0,len(df_2020_c_uni.index))

In [53]:
from geopy.distance import geodesic
df_insp['ClosestWorkingTankDis'] = 0
df_insp['ClosestWorkingTankDate'] = 'NA'
df_insp['ClosestWorkingTankConfirmNum'] = 'NA'

df_insp_Lat = df_insp['Latitude']
df_insp_Long = df_insp['Longitude']
c_shape = df_2020_c_uni.shape[0]
df_2020_c_Lat = df_2020_c_uni['LATITUDE']
df_2020_c_Long = df_2020_c_uni['LONGITUDE']
df_2020_c_Date = df_2020_c_uni['INSPECTION_DATE']
df_2020_c_ConfirmNum = df_2020_c_uni['CONFIRMATION_NUM']

ClosestDis = []
ClosestDate = []
ClosestConfirmNum = []

for i in range(df_insp.shape[0]):
    if(i%10000 == 0):
        print(i)
    loc1 = df_insp_Lat[i], df_insp_Long[i]
    dis = []
    for j in range(c_shape):
        loc2 = (df_2020_c_Lat[j], df_2020_c_Long[j])
        tem_dis = geodesic(loc1, loc2).miles
        dis.append(tem_dis)
    index = np.argmin(dis)
    ClosestDis.append(dis[index])
    ClosestDate.append(df_2020_c_Date[index])
    ClosestConfirmNum.append(df_2020_c_ConfirmNum[index])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [54]:
df_insp['ClosestWorkingTankDis'] = ClosestDis
df_insp['ClosestWorkingTankDate'] = ClosestDate
df_insp['ClosestWorkingTankConfirmNum'] = ClosestConfirmNum

In [55]:
df_insp['ClosestBadTankDis'] = 0
df_insp['ClosestBadTankDate'] = 'NA'
df_insp['ClosestBadTankConfirmNum'] = 'NA'

df_bad_tank = water_tank_clean[water_tank_clean.MEET_STANDARDS == 'N']
df_bad_tank_sorted = df_bad_tank.sort_values(by = ['INSPECTION_DATE'])
dup_bad = df_bad_tank_sorted.duplicated(subset=['LATITUDE'], keep='first')
ind_bad = [not i for i in dup_bad]
df_bad_tank_uni = df_bad_tank_sorted[ind_bad].sort_values(by = ['INSPECTION_DATE'])
df_bad_tank_uni.index = np.arange(0,len(df_bad_tank_uni.index))

df_insp_Lat = df_insp['Latitude']
df_insp_Long = df_insp['Longitude']
bad_tank_shape = df_bad_tank_uni.shape[0]
df_2020_c_Bad_Lat = df_bad_tank_uni['LATITUDE']
df_2020_c_Bad_Long = df_bad_tank_uni['LONGITUDE']
df_2020_c_Bad_Date = df_bad_tank_uni['INSPECTION_DATE']
df_2020_c_Bad_ConfirmNum = df_bad_tank_uni['CONFIRMATION_NUM']


In [56]:
ClosestBadDis = []
ClosestBadDate = []
ClosestBadConfirmNum = []
for i in range(df_insp.shape[0]):
    loc1 = df_insp_Lat[i], df_insp_Long[i]
    dis = []
    for j in range(bad_tank_shape):
        loc2 = (df_2020_c_Bad_Lat[j], df_2020_c_Bad_Long[j])
        tem_dis = geodesic(loc1, loc2).miles
        dis.append(tem_dis)
    index = np.argmin(dis)
    ClosestBadDis.append(dis[index])
    ClosestBadDate.append(df_2020_c_Bad_Date[index])
    ClosestBadConfirmNum.append(df_2020_c_Bad_ConfirmNum[index])

In [60]:
df_insp['ClosestBadTankDis'] = ClosestBadDis
df_insp['ClosestBadTankDate'] = ClosestBadDate
df_insp['ClosestBadTankConfirmNum'] = ClosestBadConfirmNum

In [61]:
#df_insp.to_csv('df_insp_with_distance-v1.csv')

In [65]:
df_insp.iloc[:,0:-1].to_csv('df_insp_with_distance-v2.csv')